In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from yellowbrick.cluster import KElbowVisualizer
import joblib

#Data di imputable pakai median

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("canggih/jog-description-and-salary-in-indonesia")

print("Path to dataset files:", path)

100%|██████████| 27.9M/27.9M [00:01<00:00, 22.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/canggih/jog-description-and-salary-in-indonesia/versions/1


In [ ]:
df_train = pd.read_csv(f'{path}/train.csv', sep='|')
df_test = pd.read_csv(f'{path}/test.csv', sep='|')

In [ ]:
df_train.head()

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,company_process_time,company_size,company_industry,job_description,salary
0,1,Facility Maintenance & Smart Warehouse Manager,Bandung,IDR,Manajer/Asisten Manajer,5 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Manufaktur,Pemeliharaan",NaN,NaN,NaN,NaN,Deskripsi PekerjaanRequirements :D3/SI from re...,NaN
1,2,Procurement Department Head,Jakarta Raya,IDR,Manajer/Asisten Manajer,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Manufaktur,Pembelian/Manajemen Material",NaN,25 days,51 - 200 pekerja,Manajemen/Konsulting HR,Job Role: 1. Responsible for material availabi...,NaN
2,3,SALES ADMIN,Jakarta Barat,IDR,Supervisor/Koordinator,4 tahun,Sarjana (S1),Penuh Waktu,"Penjualan / Pemasaran,Penjualan Ritel","Waktu regular, Senin - Jumat;Bisnis (contoh: K...",30 days,51 - 200 pekerja,Umum & Grosir,Internal Sales & AdminJob Description :We are ...,NaN
3,4,City Operation Lead Shopee Express (Cirebon),Cirebon,IDR,Supervisor/Koordinator,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Pelayanan,Logistik/Rantai Pasokan","Tip;Waktu regular, Senin - Jumat;Kasual (conto...",21 days,2001 - 5000 pekerja,Retail/Merchandise,Job Description:Responsible for HSE implementa...,NaN
4,5,Japanese Interpreter,Bekasi,IDR,Pegawai (non-manajemen & non-supervisor),2 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Lainnya,Jurnalis/Editor",NaN,23 days,201 - 500 pekerja,Manajemen/Konsulting HR,Overview: Our clients is manufacture for autom...,NaN


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31746 entries, 0 to 31745
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    31746 non-null  int64  
 1   job_title             31746 non-null  object 
 2   location              31746 non-null  object 
 3   salary_currency       31742 non-null  object 
 4   career_level          31746 non-null  object 
 5   experience_level      27454 non-null  object 
 6   education_level       31746 non-null  object 
 7   employment_type       30402 non-null  object 
 8   job_function          31746 non-null  object 
 9   job_benefits          25078 non-null  object 
 10  company_process_time  22601 non-null  object 
 11  company_size          26582 non-null  object 
 12  company_industry      30232 non-null  object 
 13  job_description       31745 non-null  object 
 14  salary                6352 non-null   float64
dtypes: float64(1), int6

In [ ]:
df_train.describe()

,id,salary
count,31746.000000,6.352000e+03
mean,15873.500000,7.102566e+06
std,9164.425159,5.935238e+06
min,1.000000,1.000000e+01
25%,7937.250000,4.250000e+06
50%,15873.500000,5.500000e+06
75%,23809.750000,8.000000e+06
max,31746.000000,1.150000e+08


In [ ]:
print("\nMissing values:")
print(df_train.isnull().sum())


Missing values:
id                          0
job_title                   0
location                    0
salary_currency             4
career_level                0
experience_level         4292
education_level             0
employment_type          1344
job_function                0
job_benefits             6668
company_process_time     9145
company_size             5164
company_industry         1514
job_description             1
salary                  25394
dtype: int64


In [ ]:
total_rows = len(df_train)
missing_percentage = (df_train.isnull().sum() / total_rows * 100).sort_values(ascending=False)
print(missing_percentage)

salary                  79.991180
company_process_time    28.806779
job_benefits            21.004221
company_size            16.266616
experience_level        13.519814
company_industry         4.769105
employment_type          4.233604
salary_currency          0.012600
job_description          0.003150
education_level          0.000000
location                 0.000000
id                       0.000000
career_level             0.000000
job_title                0.000000
job_function             0.000000
dtype: float64


In [ ]:
print("Jumlah total baris:", len(df_train))
print("Jumlah salary terisi:", df_train['salary'].notna().sum())
print("Jumlah salary kosong:", df_train['salary'].isna().sum())

print("\nStatistik salary (yang tidak kosong):")
df_train['salary'].describe()

Jumlah total baris: 31746
Jumlah salary terisi: 6352
Jumlah salary kosong: 25394

Statistik salary (yang tidak kosong):


,salary
count,6.352000e+03
mean,7.102566e+06
std,5.935238e+06
min,1.000000e+01
25%,4.250000e+06
50%,5.500000e+06
75%,8.000000e+06
max,1.150000e+08


In [ ]:
# STEP 2: Membersihkan salary yang tidak masuk akal

df_train_clean = df_train.copy()

# Set salary tidak masuk akal menjadi NaN
df_train_clean.loc[
    (df_train_clean['salary'] < 1_000_000) |
    (df_train_clean['salary'] > 50_000_000),
    'salary'
] = None

# Cek ulang statistik
print("Jumlah salary terisi setelah cleaning:", df_train_clean['salary'].notna().sum())
print("Jumlah salary kosong setelah cleaning:", df_train_clean['salary'].isna().sum())

df_train_clean['salary'].describe()

Jumlah salary terisi setelah cleaning: 6322
Jumlah salary kosong setelah cleaning: 25424


,salary
count,6.322000e+03
mean,6.945058e+06
std,4.738261e+06
min,1.050000e+06
25%,4.250000e+06
50%,5.500000e+06
75%,8.000000e+06
max,5.000000e+07


In [ ]:
# STEP 3: Hitung nilai dummy salary (median)

dummy_salary = df_train_clean['salary'].median()

print("Nilai dummy salary (median):", dummy_salary)

Nilai dummy salary (median): 5500000.0


In [ ]:
# STEP 4: Menambahkan flag imputasi

df_train_clean['salary_imputed'] = df_train_clean['salary'].isna().astype(int)

# Cek distribusi flag
df_train_clean['salary_imputed'].value_counts()

,count
salary_imputed,
1,25424
0,6322


In [ ]:
# STEP 5: Mengisi salary kosong dengan dummy

df_train_clean['salary'] = df_train_clean['salary'].fillna(dummy_salary)

# Verifikasi
print("Jumlah salary kosong setelah imputasi:", df_train_clean['salary'].isna().sum())
df_train_clean['salary'].describe()

Jumlah salary kosong setelah imputasi: 0


,salary
count,3.174600e+04
mean,5.787773e+06
std,2.191681e+06
min,1.050000e+06
25%,5.500000e+06
50%,5.500000e+06
75%,5.500000e+06
max,5.000000e+07


In [ ]:
# STEP 6: Final check
df_train_clean.info()

print("\nCrosstab salary_imputed:")
pd.crosstab(df_train_clean['salary_imputed'], columns="count")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31746 entries, 0 to 31745
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    31746 non-null  int64  
 1   job_title             31746 non-null  object 
 2   location              31746 non-null  object 
 3   salary_currency       31742 non-null  object 
 4   career_level          31746 non-null  object 
 5   experience_level      27454 non-null  object 
 6   education_level       31746 non-null  object 
 7   employment_type       30402 non-null  object 
 8   job_function          31746 non-null  object 
 9   job_benefits          25078 non-null  object 
 10  company_process_time  22601 non-null  object 
 11  company_size          26582 non-null  object 
 12  company_industry      30232 non-null  object 
 13  job_description       31745 non-null  object 
 14  salary                31746 non-null  float64
 15  salary_imputed     

col_0,count
salary_imputed,
0,6322
1,25424


In [ ]:
# STEP 7: Simpan hasil imputasi ke CSV baru

output_path = "train_salary_imputed.csv"

df_train_clean.to_csv(output_path, index=False)

print(f"File berhasil disimpan sebagai: {output_path}")

File berhasil disimpan sebagai: train_salary_imputed.csv


# Coba-coba
### Dalam pembuatan model, tidak dilakukan penghapusan data secara massal.Nilai hilang pada fitur kategorikal diimputasi menggunakan mode atau label eksplisit.Kolom salary tidak diimputasi, melainkan digunakan sebagai target pada baris yang tersedia, dan diprediksi pada baris yang tidak memiliki nilai salary.

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31746 entries, 0 to 31745
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    31746 non-null  int64  
 1   job_title             31746 non-null  object 
 2   location              31746 non-null  object 
 3   salary_currency       31742 non-null  object 
 4   career_level          31746 non-null  object 
 5   experience_level      27454 non-null  object 
 6   education_level       31746 non-null  object 
 7   employment_type       30402 non-null  object 
 8   job_function          31746 non-null  object 
 9   job_benefits          25078 non-null  object 
 10  company_process_time  22601 non-null  object 
 11  company_size          26582 non-null  object 
 12  company_industry      30232 non-null  object 
 13  job_description       31745 non-null  object 
 14  salary                6352 non-null   float64
dtypes: float64(1), int6

In [ ]:
missing_summary = (
    df_train.isna()
    .sum()
    .to_frame(name="missing_count")
    .assign(
        missing_pct=lambda x: (x["missing_count"] / len(df_train)) * 100
    )
    .sort_values("missing_pct", ascending=False)
)

missing_summary

,missing_count,missing_pct
salary,25394,79.991180
company_process_time,9145,28.806779
job_benefits,6668,21.004221
company_size,5164,16.266616
experience_level,4292,13.519814
company_industry,1514,4.769105
employment_type,1344,4.233604
salary_currency,4,0.012600
job_description,1,0.003150
education_level,0,0.000000


In [ ]:
categorical_cols_mode = [
    "experience_level",
    "employment_type",
    "company_industry",
    "salary_currency",
    "job_description"
]

for col in categorical_cols_mode:
    mode_value = df_train[col].mode()[0]
    df_train[col] = df_train[col].fillna(mode_value)

In [ ]:
df_train[categorical_cols_mode].isna().sum()

,0
experience_level,0
employment_type,0
company_industry,0
salary_currency,0
job_description,0


In [ ]:
explicit_label_cols = [
    "company_process_time",
    "job_benefits",
    "company_size"
]

for col in explicit_label_cols:
    df_train[col] = df_train[col].fillna("Not Specified")

In [ ]:
df_train[explicit_label_cols].isna().sum()

,0
company_process_time,0
job_benefits,0
company_size,0


In [ ]:
df_with_salary = df_train[df_train["salary"].notna()].copy()
df_without_salary = df_train[df_train["salary"].isna()].copy()

print("Data dengan salary:", df_with_salary.shape)
print("Data tanpa salary:", df_without_salary.shape)

Data dengan salary: (6352, 15)
Data tanpa salary: (25394, 15)


In [ ]:
feature_cols = [
    "job_title",
    "experience_level",
    "career_level",
    "location",
    "employment_type",
    "company_size"
]

X = df_with_salary[feature_cols]
y = df_with_salary["salary"]

print(X.shape)
print(y.shape)

(6352, 6)
(6352,)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

print("X_train:", X_train.shape)
print("X_test :", X_test.shape)

X_train: (5081, 6)
X_test : (1271, 6)


In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(
    handle_unknown="ignore",
    sparse_output=False
)

# fit hanya pada data training
X_train_encoded = encoder.fit_transform(X_train)

# test hanya transform
X_test_encoded = encoder.transform(X_test)

print("X_train_encoded:", X_train_encoded.shape)
print("X_test_encoded :", X_test_encoded.shape)

X_train_encoded: (5081, 3758)
X_test_encoded : (1271, 3758)


In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train_encoded, y_train)

LinearRegression()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = model.predict(X_test_encoded)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("MAE :", mae)
print("RMSE:", rmse)

MAE : 2302923.2219523243
RMSE: 4909662.0043803295


In [ ]:
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

print(y_train_log.min(), y_train_log.max())

2.3978952727983707 18.5159909328476


In [ ]:
from sklearn.linear_model import LinearRegression

log_model = LinearRegression()
log_model.fit(X_train_encoded, y_train_log)

LinearRegression()

In [ ]:
# Prediksi di log-space
y_pred_log = log_model.predict(X_test_encoded)

# Kembalikan ke skala asli
y_pred = np.expm1(y_pred_log)

# Evaluasi di skala asli
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae_log = mean_absolute_error(y_test, y_pred)
rmse_log = np.sqrt(mean_squared_error(y_test, y_pred))

print("MAE (log model):", mae_log)
print("RMSE (log model):", rmse_log)

MAE (log model): 2113551.6393421288
RMSE (log model): 5092360.520878064


In [ ]:
from sklearn.linear_model import Ridge

ridge_model = Ridge(alpha=1.0, random_state=42)
ridge_model.fit(X_train_encoded, y_train_log)

Ridge(random_state=42)

In [ ]:
y_pred_log_ridge = ridge_model.predict(X_test_encoded)
y_pred_ridge = np.expm1(y_pred_log_ridge)

from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

mae_ridge = mean_absolute_error(y_test, y_pred_ridge)
rmse_ridge = np.sqrt(mean_squared_error(y_test, y_pred_ridge))

print("MAE (Ridge + log):", mae_ridge)
print("RMSE (Ridge + log):", rmse_ridge)

MAE (Ridge + log): 1954265.1683147508
RMSE (Ridge + log): 4332315.28987162


In [ ]:
X_pred = df_without_salary[feature_cols]
X_pred_encoded = encoder.transform(X_pred)

In [ ]:
salary_pred_log = ridge_model.predict(X_pred_encoded)
salary_pred = np.expm1(salary_pred_log)

df_without_salary["salary_predicted"] = salary_pred

In [ ]:
df_without_salary["salary_predicted"].describe()

,salary_predicted
count,2.539400e+04
mean,6.698720e+06
std,3.801310e+06
min,7.455366e+05
25%,4.569856e+06
50%,5.686999e+06
75%,7.526404e+06
max,4.229630e+07


# Coba - coba 2

In [ ]:
# 1. Tentukan target
target = "salary"

# 2. Kolom yang HARUS dibuang
drop_cols = ["id", "salary_imputed", "salary"]

# 3. Feature matrix (X) dan target (y)
X = df_train_clean.drop(columns=drop_cols)
y = df_train_clean[target]

# 4. Cek hasilnya
print("Shape X:", X.shape)
print("Shape y:", y.shape)
print("\nKolom X:")
print(X.columns)

Shape X: (31746, 14)
Shape y: (31746,)

Kolom X:
Index(['job_title', 'location', 'salary_currency', 'career_level',
       'experience_level', 'education_level', 'employment_type',
       'job_function', 'job_benefits', 'company_process_time', 'company_size',
       'company_industry', 'job_description', 'job_title_clean'],
      dtype='object')


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(25396, 14) (6350, 14)
(25396,) (6350,)


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, r2_score

# Baseline features
cat_features = [
    'career_level',
    'experience_level',
    'education_level',
    'employment_type',
    'company_size',
    'company_industry',
    'salary_currency',
    'company_process_time'
]

# Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
    ]
)

# Pipeline
model = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('regressor', Ridge(alpha=1.0))
])

# Train
model.fit(X_train[cat_features], y_train)

# Predict
y_pred = model.predict(X_test[cat_features])

# Evaluation
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("MAE:", mae)
print("R2 :", r2)


MAE: 935006.5880767943
R2 : 0.10780875326842898


Rata-rata kesalahan prediksi 935.000

Menangkap perbedaan UMR / pasar kerja antar lokasi, yang secara ekonomi memang memengaruhi gaji.

In [ ]:
# Jumlah unique location
n_unique = X_train['location'].nunique()
print("Jumlah unique location:", n_unique)

# Top 15 lokasi paling sering
X_train['location'].value_counts().head(15)


Jumlah unique location: 193


,count
location,
Jakarta Raya,6082
Jakarta Selatan,1881
Tangerang,1400
Jakarta Barat,1392
Jakarta Utara,1164
Jakarta Pusat,964
Surabaya,939
Bandung,667
Bekasi,546


In [ ]:
# Feature set + location
cat_features_loc = [
    'career_level',
    'experience_level',
    'education_level',
    'employment_type',
    'company_size',
    'company_industry',
    'salary_currency',
    'company_process_time',
    'location'
]

# Preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features_loc)
    ]
)

# Pipeline
model_loc = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('regressor', Ridge(alpha=1.0))
])

# Train
model_loc.fit(X_train[cat_features_loc], y_train)

# Predict
y_pred_loc = model_loc.predict(X_test[cat_features_loc])

# Evaluate
mae_loc = mean_absolute_error(y_test, y_pred_loc)
r2_loc = r2_score(y_test, y_pred_loc)

print("MAE (with location):", mae_loc)
print("R2  (with location):", r2_loc)

MAE (with location): 951146.4471465243
R2  (with location): 0.11286338558740239


In [ ]:
# Jumlah unique job title
n_unique_title = X_train['job_title'].nunique()
print("Jumlah unique job_title:", n_unique_title)

# Distribusi frekuensi job title
X_train['job_title'].value_counts().describe()

Jumlah unique job_title: 15989


,count
count,15989.000000
mean,1.588342
std,3.508135
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,229.000000


In [ ]:
# Jumlah unique job title
n_unique_title = X_train['job_title'].nunique()
print("Jumlah unique job_title:", n_unique_title)

# Distribusi frekuensi job title
X_train['job_title'].value_counts().describe()

X_train['job_title'].value_counts().head(15)

Jumlah unique job_title: 15989


,count
job_title,
Sales Executive,229
Management Trainee,127
Sales,91
Accounting Staff,91
Sales Engineer,90
Marketing,75
Graphic Designer,67
Digital Marketing,62
Sales Marketing,59


In [ ]:
# Hitung frekuensi job_title di TRAIN
job_title_counts = X_train['job_title'].value_counts()

# Ambil job_title yang cukup sering
frequent_titles = job_title_counts[job_title_counts >= 50].index

print("Jumlah job_title dipertahankan:", len(frequent_titles))

Jumlah job_title dipertahankan: 13


In [ ]:
# Replace job_title langka jadi 'Other'
X_train['job_title_clean'] = X_train['job_title'].where(
    X_train['job_title'].isin(frequent_titles),
    'Other'
)

X_test['job_title_clean'] = X_test['job_title'].where(
    X_test['job_title'].isin(frequent_titles),
    'Other'
)

# Cek hasil
print(X_train['job_title_clean'].value_counts().head(15))

job_title_clean
Other                 24297
Sales Executive         229
Management Trainee      127
Accounting Staff         91
Sales                    91
Sales Engineer           90
Marketing                75
Graphic Designer         67
Digital Marketing        62
Sales Marketing          59
Content Creator          54
Account Executive        53
Accounting               51
Project Manager          50
Name: count, dtype: int64


In [ ]:
cat_features_final = [
    'career_level',
    'experience_level',
    'education_level',
    'employment_type',
    'company_size',
    'company_industry',
    'salary_currency',
    'company_process_time',
    'location',
    'job_title_clean'
]

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features_final)
    ]
)

model_final = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('regressor', Ridge(alpha=1.0))
])

# Train
model_final.fit(X_train[cat_features_final], y_train)

# Predict
y_pred_final = model_final.predict(X_test[cat_features_final])

# Evaluate
mae_final = mean_absolute_error(y_test, y_pred_final)
r2_final = r2_score(y_test, y_pred_final)

print("FINAL MAE:", mae_final)
print("FINAL R2 :", r2_final)

FINAL MAE: 948284.0475615544
FINAL R2 : 0.11335446521867132


#coba-coba mengubah imputable dummy data sebelumnya dengan lokasi , pengalaman kerja , dan job title

In [ ]:
df_train_clean['salary_imputed'].value_counts()

,count
salary_imputed,
1,25424
0,6322


In [ ]:
df_ref = df_train_clean[df_train_clean['salary_imputed'] == 0].copy()

print("Jumlah data salary asli:", df_ref.shape[0])
df_ref[['salary']].describe()

Jumlah data salary asli: 6322


,salary
count,6.322000e+03
mean,6.945058e+06
std,4.738261e+06
min,1.050000e+06
25%,4.250000e+06
50%,5.500000e+06
75%,8.000000e+06
max,5.000000e+07


In [ ]:
# Re-generate job_title_clean di seluruh dataframe (BERDASARKAN TRAIN)
job_title_counts = X_train['job_title'].value_counts()
frequent_titles = job_title_counts[job_title_counts >= 50].index

df_train_clean['job_title_clean'] = df_train_clean['job_title'].where(
    df_train_clean['job_title'].isin(frequent_titles),
    'Other'
)

# Validasi
df_train_clean['job_title_clean'].value_counts().head(10)

,count
job_title_clean,
Other,31473
Sales Executive,273


In [ ]:
df_ref = df_train_clean[df_train_clean['salary_imputed'] == 0].copy()

print("Jumlah data referensi:", df_ref.shape[0])

Jumlah data referensi: 6322


In [ ]:
median_jle = (
    df_ref
    .groupby(['job_title_clean', 'location', 'experience_level'])['salary']
    .median()
    .reset_index()
    .rename(columns={'salary': 'median_jle'})
)

In [ ]:
median_jl = (
    df_ref
    .groupby(['job_title_clean', 'location'])['salary']
    .median()
    .reset_index()
    .rename(columns={'salary': 'median_jl'})
)

In [ ]:
median_je = (
    df_ref
    .groupby(['job_title_clean', 'experience_level'])['salary']
    .median()
    .reset_index()
    .rename(columns={'salary': 'median_je'})
)

In [ ]:
median_j = (
    df_ref
    .groupby(['job_title_clean'])['salary']
    .median()
    .reset_index()
    .rename(columns={'salary': 'median_j'})
)

In [ ]:
df_imp = df_train_clean.copy()

df_imp = df_imp.merge(median_jle, on=['job_title_clean', 'location', 'experience_level'], how='left')
df_imp = df_imp.merge(median_jl,  on=['job_title_clean', 'location'], how='left')
df_imp = df_imp.merge(median_je,  on=['job_title_clean', 'experience_level'], how='left')
df_imp = df_imp.merge(median_j,   on=['job_title_clean'], how='left')

In [ ]:
mask = df_imp['salary_imputed'] == 1
global_median = df_ref['salary'].median()

df_imp.loc[mask, 'salary_dummy'] = (
    df_imp.loc[mask, 'median_jle']
    .fillna(df_imp.loc[mask, 'median_jl'])
    .fillna(df_imp.loc[mask, 'median_je'])
    .fillna(df_imp.loc[mask, 'median_j'])
    .fillna(global_median)
)

In [ ]:
print("SALARY IMPUTASI LAMA:")
df_train_clean.loc[mask, 'salary'].describe()


SALARY IMPUTASI LAMA:


,salary
count,25424.0
mean,5500000.0
std,0.0
min,5500000.0
25%,5500000.0
50%,5500000.0
75%,5500000.0
max,5500000.0


In [ ]:
print("\nSALARY DUMMY BARU:")
df_imp.loc[mask, 'salary_dummy'].describe()


SALARY DUMMY BARU:


,salary_dummy
count,2.542400e+04
mean,6.599678e+06
std,3.534002e+06
min,2.160000e+06
25%,4.700000e+06
50%,5.550000e+06
75%,6.625000e+06
max,4.800000e+07


In [ ]:
df_imp['salary_final'] = df_imp['salary'].where(
    df_imp['salary_imputed'] == 0,
    df_imp['salary_dummy']
)

In [ ]:
# Kolom yang TIDAK perlu disimpan
drop_audit_cols = [
    'median_jle', 'median_jl', 'median_je', 'median_j',
    'salary_dummy'
]

df_final = df_imp.drop(columns=drop_audit_cols, errors='ignore')

df_final.head()

,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,company_process_time,company_size,company_industry,job_description,salary,salary_imputed,job_title_clean,salary_final
0,1,Facility Maintenance & Smart Warehouse Manager,Bandung,IDR,Manajer/Asisten Manajer,5 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Manufaktur,Pemeliharaan",NaN,NaN,NaN,NaN,Deskripsi PekerjaanRequirements :D3/SI from re...,5500000.0,1,Other,7500000.0
1,2,Procurement Department Head,Jakarta Raya,IDR,Manajer/Asisten Manajer,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Manufaktur,Pembelian/Manajemen Material",NaN,25 days,51 - 200 pekerja,Manajemen/Konsulting HR,Job Role: 1. Responsible for material availabi...,5500000.0,1,Other,13250000.0
2,3,SALES ADMIN,Jakarta Barat,IDR,Supervisor/Koordinator,4 tahun,Sarjana (S1),Penuh Waktu,"Penjualan / Pemasaran,Penjualan Ritel","Waktu regular, Senin - Jumat;Bisnis (contoh: K...",30 days,51 - 200 pekerja,Umum & Grosir,Internal Sales & AdminJob Description :We are ...,5500000.0,1,Other,19550000.0
3,4,City Operation Lead Shopee Express (Cirebon),Cirebon,IDR,Supervisor/Koordinator,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Pelayanan,Logistik/Rantai Pasokan","Tip;Waktu regular, Senin - Jumat;Kasual (conto...",21 days,2001 - 5000 pekerja,Retail/Merchandise,Job Description:Responsible for HSE implementa...,5500000.0,1,Other,13500000.0
4,5,Japanese Interpreter,Bekasi,IDR,Pegawai (non-manajemen & non-supervisor),2 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Lainnya,Jurnalis/Editor",NaN,23 days,201 - 500 pekerja,Manajemen/Konsulting HR,Overview: Our clients is manufacture for autom...,5500000.0,1,Other,5025000.0


In [ ]:
df_final['salary_final'] = df_final['salary_final'].round(0).astype(int)

In [ ]:
csv_path = "salary_dataset_clean_final.csv"
df_final.to_csv(csv_path, index=False)

print(f"File CSV tersimpan: {csv_path}")

File CSV tersimpan: salary_dataset_clean_final.csv
